In [ ]:
import pandas as pd

In [ ]:
#df=pd.read_csv('news_data.csv',encoding='latin1')

df=pd.read_csv('news_data.csv',encoding='unicode_escape')

df.head()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
###Drop Nan Values
df=df.dropna()

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [ ]:
  ## Get the Dependent features
y=df['label']

In [ ]:
X.shape

In [ ]:
y.shape

In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
### Vocabulary size
voc_size=5000

###$\color{blue}{\text{Onehot Representation}}$

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

In [ ]:
messages

In [ ]:
#messages.reset_index(inplace=True)
#messages

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

In [ ]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
corpus

In [ ]:
corpus[2]

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

In [ ]:
corpus[2]

In [ ]:
onehot_repr[2]

###$\color{blue}{\text{Embedding Representation}}$

In [ ]:
sent_length=20 # sentence lenght
#embedded_docs=pad_sequences(onehot_repr,padding='post',maxlen=sent_length)
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

In [ ]:
embedded_docs[2]

In [ ]:
embedded_docs[5]

In [ ]:
## Creating model
embedding_vector_features=40 ##features representation every word with 40 features
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

In [ ]:
len(embedded_docs),y.shape

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

###$\color{blue}{\text{Model Training}}$

In [ ]:
# Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

###$\color{blue}{\text{Adding Dropout }}$

In [ ]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

###$\color{blue}{\text{Performance Metrics And Accuracy }}$

In [ ]:
y_pred=model.predict(X_test)

In [ ]:
y_pred=np.where(y_pred > 0.6, 1,0) ##AUC ROC Curve

In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test,y_pred))